In [1]:
import tkinter as tk
from tkinter import messagebox

class MiceDistributorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Mice Distributor")

        # Data structures
        self.mice_data = []  # Will hold tuples of (mouse_id, weight)

        # 1) Number of Groups
        self.lbl_groups = tk.Label(root, text="Number of Groups:")
        self.lbl_groups.grid(row=0, column=0, padx=5, pady=5, sticky=tk.E)
        self.entry_groups = tk.Entry(root)
        self.entry_groups.grid(row=0, column=1, padx=5, pady=5)

        # 2) Fields for Mouse ID and Weight
        self.lbl_mouse_id = tk.Label(root, text="Mouse ID:")
        self.lbl_mouse_id.grid(row=1, column=0, padx=5, pady=5, sticky=tk.E)
        self.entry_mouse_id = tk.Entry(root)
        self.entry_mouse_id.grid(row=1, column=1, padx=5, pady=5)

        self.lbl_weight = tk.Label(root, text="Weight:")
        self.lbl_weight.grid(row=2, column=0, padx=5, pady=5, sticky=tk.E)
        self.entry_weight = tk.Entry(root)
        self.entry_weight.grid(row=2, column=1, padx=5, pady=5)

        # 3) Buttons
        self.btn_add_mouse = tk.Button(root, text="Add Mouse", command=self.add_mouse)
        self.btn_add_mouse.grid(row=3, column=0, padx=5, pady=5, sticky=tk.E)

        self.btn_distribute = tk.Button(root, text="Distribute", command=self.distribute)
        self.btn_distribute.grid(row=3, column=1, padx=5, pady=5, sticky=tk.W)

        # 4) Results (Text Widget to display output)
        self.text_results = tk.Text(root, width=60, height=15)
        self.text_results.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

    def add_mouse(self):
        """
        Adds a (mouse_id, weight) entry to the self.mice_data list.
        """
        mouse_id = self.entry_mouse_id.get().strip()
        weight_str = self.entry_weight.get().strip()

        if not mouse_id or not weight_str:
            messagebox.showerror("Input Error", "Please enter both Mouse ID and Weight.")
            return

        # Validate weight is float
        try:
            weight = float(weight_str)
        except ValueError:
            messagebox.showerror("Input Error", "Weight must be a number.")
            return

        # Add to list
        self.mice_data.append((mouse_id, weight))

        # Clear the entry fields
        self.entry_mouse_id.delete(0, tk.END)
        self.entry_weight.delete(0, tk.END)

        # Show a small confirmation or just print to results text
        self.text_results.insert(tk.END, f"Added: Mouse ID={mouse_id}, Weight={weight}\n")

    def distribute(self):
        """
        Distribute the mice into the specified number of groups using a greedy approach.
        Then display the results in the text widget.
        """
        # First, read how many groups the user wants
        n_groups_str = self.entry_groups.get().strip()
        if not n_groups_str:
            messagebox.showerror("Input Error", "Please enter number of groups.")
            return

        try:
            n_groups = int(n_groups_str)
            if n_groups <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("Input Error", "Number of groups must be a positive integer.")
            return

        if len(self.mice_data) == 0:
            messagebox.showerror("No Data", "No mice data to distribute.")
            return

        # Perform the distribution
        groups = self.distribute_mice_by_weight(self.mice_data, n_groups)

        # Clear the text widget before showing new results
        self.text_results.delete("1.0", tk.END)

        # Calculate total or average weights
        group_weights = [sum(m[1] for m in group) for group in groups]
        group_sizes = [len(group) for group in groups]
        group_averages = [
            (group_weights[i]/group_sizes[i] if group_sizes[i] > 0 else 0)
            for i in range(n_groups)
        ]

        # Display group info
        for i, group in enumerate(groups, start=1):
            self.text_results.insert(tk.END, f"--- Group {i} ---\n")
            for mouse_id, w in group:
                self.text_results.insert(tk.END, f"  Mouse ID: {mouse_id}, Weight: {w}\n")
            self.text_results.insert(tk.END, f"  Total Weight: {group_weights[i-1]:.2f}\n")
            self.text_results.insert(tk.END, f"  Average Weight: {group_averages[i-1]:.2f}\n\n")

        # Show difference in total or average
        min_total, max_total = min(group_weights), max(group_weights)
        diff_total = max_total - min_total
        self.text_results.insert(tk.END, f"Difference between lightest and heaviest group (Total): {diff_total:.2f}\n")

        min_avg, max_avg = min(group_averages), max(group_averages)
        diff_avg = max_avg - min_avg
        self.text_results.insert(tk.END, f"Difference between lightest and heaviest group (Average): {diff_avg:.2f}\n")

    def distribute_mice_by_weight(self, mice, n_groups):
        """
        Greedy approach to distribute mice into n groups based on body weight.
        Sort by descending weight, then place each mouse in the currently lightest group so far.
        Returns a list of groups, where each group is a list of (mouse_id, weight) tuples.
        """
        # Sort descending by weight
        mice_sorted = sorted(mice, key=lambda x: x[1], reverse=True)

        # Initialize groups and sums
        groups = [[] for _ in range(n_groups)]
        sums = [0.0]*n_groups

        # Distribute each mouse
        for mouse in mice_sorted:
            # Find the index of the group with the smallest total so far
            idx = sums.index(min(sums))
            groups[idx].append(mouse)
            sums[idx] += mouse[1]

        return groups

def main():
    root = tk.Tk()
    app = MiceDistributorApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
